In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import TrainingArguments, Trainer

In [ ]:
!pip install gdown -q 
!pip install transformers -q
!pip install simplet5 -q
!gdown 1tHwLTAgI_cRFr4y50MBipDPbFXayro0Q
!pip install wandb

Downloading...
From: https://drive.google.com/uc?id=1tHwLTAgI_cRFr4y50MBipDPbFXayro0Q
To: /content/news_summary.csv
100% 11.9M/11.9M [00:00<00:00, 187MB/s]
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 49.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.7/201.7 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.1 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8808 sha256=5e35971e79ade59e3868b55860b7cb6c4289862ed2e39e2adb09987c02363cc5
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [ ]:
%env WANDB_PROJECT=amazon_sentiment_analysis

env: WANDB_PROJECT=amazon_sentiment_analysis


In [2]:
df = pd.read_csv("news_summary.csv",encoding="latin-1")
df.head()

,author,date,headlines,read_more,text,ctext
0,Chhavi Tyagi,"03 Aug 2017,Thursday",Daman & Diu revokes mandatory Rakshabandhan in...,http://www.hindustantimes.com/india-news/raksh...,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Daisy Mowke,"03 Aug 2017,Thursday",Malaika slams user who trolled her for 'divorc...,http://www.hindustantimes.com/bollywood/malaik...,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,Arshiya Chopra,"03 Aug 2017,Thursday",'Virgin' now corrected to 'Unmarried' in IGIMS...,http://www.hindustantimes.com/patna/bihar-igim...,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Sumedha Sehra,"03 Aug 2017,Thursday",Aaj aapne pakad liya: LeT man Dujana before be...,http://indiatoday.intoday.in/story/abu-dujana-...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Aarushi Maheshwari,"03 Aug 2017,Thursday",Hotel staff to get training to spot signs of s...,http://indiatoday.intoday.in/story/sex-traffic...,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...


In [3]:
df=df[["text","ctext"]]
print(df.shape)
df.head()

(4514, 2)


,text,ctext
0,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...


In [4]:
df.columns = ["target_text","source_text"] #simpleT5 expects its input&output to be specified as such.
df=df.dropna()
print(df.shape)
df.head()



(4396, 2)


,target_text,source_text
0,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...


In [5]:
mySum=df.iat[0,0]
myText=df.iat[0,1]
print("Original text : "+ myText + "\nLength : "+str(len(myText)))
print("Summary : "+ mySum + "\nLength : "+str(len(mySum)))

Original text : The Daman and Diu administration on Wednesday withdrew a circular that asked women staff to tie rakhis on male colleagues after the order triggered a backlash from employees and was ripped apart on social media.The union territory?s administration was forced to retreat within 24 hours of issuing the circular that made it compulsory for its staff to celebrate Rakshabandhan at workplace.?It has been decided to celebrate the festival of Rakshabandhan on August 7. In this connection, all offices/ departments shall remain open and celebrate the festival collectively at a suitable time wherein all the lady staff shall tie rakhis to their colleagues,? the order, issued on August 1 by Gurpreet Singh, deputy secretary (personnel), had said.To ensure that no one skipped office, an attendance report was to be sent to the government the next evening.The two notifications ? one mandating the celebration of Rakshabandhan (left) and the other withdrawing the mandate (right) ? were iss

In [6]:
s=0
ss=0
for v in range(4396) :
  s=s+len(df.iat[v,1])
  ss=ss+len(df.iat[v,0])
print("average article length : "+ str(s/4396)+"\naverage summary legth :"+str(ss/4396))

average article length : 2088.4374431301185
average summary legth :354.82074613284806


In [7]:
# maxArt=0
# maxSum=0
# pos=0
# for i in range(4368) :
#   src=len(df.iat[i,1].split())
#   sum=len(df.iat[i,0].split())
#   if src>maxArt :
#     maxArt=src
#     pos=i
#   if maxSum<sum :
#     maxSum=sum
#     pos2=i
# print(maxArt,maxSum)
# print(pos,pos2)



In [8]:
def check_length (row) :
  return len(row["source_text"].split())<=2650

In [9]:
keep_rows=df.apply(check_length,axis=1)

In [10]:
len(keep_rows)
df=df[keep_rows]


In [11]:
df['source_text'] = "summarize: " + df['source_text'] #T5 model expects a prefix related to each task so we will add "summarize:"" since its a summarization task."
df.head()

,target_text,source_text
0,The Administration of Union Territory Daman an...,summarize: The Daman and Diu administration on...
1,Malaika Arora slammed an Instagram user who tr...,summarize: From her special numbers to TV?appe...
2,The Indira Gandhi Institute of Medical Science...,summarize: The Indira Gandhi Institute of Medi...
3,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,summarize: Lashkar-e-Taiba's Kashmir commander...
4,Hotels in Maharashtra will train their staff t...,summarize: Hotels in Mumbai and other Indian c...


In [13]:
example_art=df.iat[0,1]

In [14]:
example_art

'summarize: The Daman and Diu administration on Wednesday withdrew a circular that asked women staff to tie rakhis on male colleagues after the order triggered a backlash from employees and was ripped apart on social media.The union territory?s administration was forced to retreat within 24 hours of issuing the circular that made it compulsory for its staff to celebrate Rakshabandhan at workplace.?It has been decided to celebrate the festival of Rakshabandhan on August 7. In this connection, all offices/ departments shall remain open and celebrate the festival collectively at a suitable time wherein all the lady staff shall tie rakhis to their colleagues,? the order, issued on August 1 by Gurpreet Singh, deputy secretary (personnel), had said.To ensure that no one skipped office, an attendance report was to be sent to the government the next evening.The two notifications ? one mandating the celebration of Rakshabandhan (left) and the other withdrawing the mandate (right) ? were issued 

In [12]:
train_df, test_df = train_test_split(df, test_size=0.2)
train_df.shape, test_df.shape

((3505, 2), (877, 2))

In [ ]:
from simplet5 import SimpleT5
import os

model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


In [ ]:
source_max_token_len = 2200
target_max_token_len = 200
batch_size = 8
max_epochs = 5

r=model.train(train_df=train_df,
              eval_df=test_df,
              source_max_token_len=source_max_token_len,
              target_max_token_len=target_max_token_len,
              batch_size=batch_size,
              max_epochs=max_epochs,
              use_gpu=True
              )

TypeError: ignored

In [15]:
from simplet5 import SimpleT5

# Initialize SimpleT5
model = SimpleT5()

# Load the model from checkpoint
# Make sure to replace "checkpoints/epoch=4-step=999.ckpt" with your actual checkpoint path
model.load_model("t5", r"D:\simplet5-epoch-1-train-loss-1.3686-val-loss-1.387")

# Now use the model to make predictions.
# Replace "Some input text" with your actual input
input_text = example_art
prediction = model.predict(input_text)

# Print the prediction
print(prediction)



Global seed set to 42
Token indices sequence length is longer than the specified maximum sequence length for this model (545 > 512). Running this sequence through the model will result in indexing errors


['The Daman and Diu administration on Wednesday withdrew a circular that made it compulsory for women staff to tie rakhis on male colleagues after the order was ripped apart on social media. "It has been decided to celebrate the festival of Rakshabandhan on August 7, in this connection, all offices/ departments shall remain open and celebrate the festival collectively at a suitable time wherein all the lady staff shall tie rakhis to their colleagues," said the administration.']
